# Preface
> The questions/exercises given for the assignment has been repeated in this notebook, they have been put in bold. Our answers and explanations have been indented, or printed from the executed code, to easily distinguish them from the exercise text. Throughout the text we are using good guys and heroes synonymously and bad guys and villains synonymously. We also use the word alignment(s) to refer to good and bad collectively.

## Imports
> The following imports are needed throughout the notebook:

In [ ]:
import csv
import os
import nltk
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mpl
import re
import string
import statistics
import numpy as np
import random
import math
import pandas as pd
import json
import urllib.request
import powerlaw

from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from wordcloud import WordCloud
from community import community_louvain
from collections import Counter
from os import listdir
from bs4 import BeautifulSoup
from fa2 import ForceAtlas2

## Data

> The downloaded files can be found in the folders `dc` and `marvel`, in our [github project](https://github.com/pete414n/02805SG/tree/main/assignment2).

# Basic Stats

**Write a short paragraph describing the network. The paragraph should contain the following information**
- **The number of nodes and links.**

In [ ]:
#Edgelist saved as gml so we can keep the node attributes
G = nx.read_gml("superhero.edgelist")

In [ ]:
# Number of nodes
num_nodes = len(G.nodes())
print(f"Number of nodes = {num_nodes}")

# Number of links
num_links = len(G.edges()) # Assuming we get an edge list
print(f"Number of links = {num_links}")

- **The average, median, mode, minimum and maximum value of the network's in-degree.s And of the out-degrees.**

In [ ]:
# Making a sorted list of the in degrees and a list of the out degrees
in_degree_list = sorted([d for (c, d) in G.in_degree], reverse=True)
out_degree_list = sorted([d for (c, d) in G.out_degree], reverse=True)

# Average in degree
print(f"average in degree = {np.mean(in_degree_list)}")

# Average out degree
print(f"average out degree = {np.mean(out_degree_list)}")

In [ ]:
# Median in degree
print(f"median in degree = {np.median(in_degree_list)}")

# Median out degree
print(f"median out degree = {np.median(out_degree_list)}")

In [ ]:
# Mode in degree
mode = max(set(in_degree_list), key=in_degree_list.count)
print(f"mode in degree = {mode}")

# Mode out degree
mode = max(set(out_degree_list), key=out_degree_list.count)
print(f"mode out degree = {mode}")

In [ ]:
# Since we have the sorted list, the first element will be the max and the last will be the min
# Minimum and maximum in degree
min_in_degree = in_degree_list[-1]
max_in_degree = in_degree_list[0]
print(f"minimum in degree = {min_in_degree}\nmaximum in degree = {max_in_degree}\n")

# Minimum and maximum out degree
min_out_degree = out_degree_list[-1]
max_out_degree = out_degree_list[0]
print(f"minimum out degree = {min_out_degree}\nmaximum out degree = {max_out_degree}\n")

> When creating the network we have tried to get the wikitext as clean as possible and only have the true text without i.e. the contents list. 
We have created the network so the nodes conatins 3 attributes, universe, txtlength and sentiment. The first one tells whether it's a DC or Marvel character, the next gives the length of the whole wikitext and the last gives the sentiment value of the wikitext. 
We have chosen not to include all the characters with pages that were redirects. We have done this by checking if the text belonging to the given character were starting with #redirect, if so we didn't include this node in the network. We have also removed all nodes who had a sentiment value of 0. Then we have removed isolates from the network and found the largest connected component. 

> Our network has 1602 nodes and 19068. The average in- and out-degree are both 11.75..., they are the same since each edge contributes one to the total out-degree and one to the total in-degree, and a difference would indicate that there somehow where a higher total in-degree than a total out-degree, or vice versa. The median in degree is 5, and the median out degree is 9. The mode for in degree is 0, which corresponds to characters that link to other characters, but are not linked to by anyone else. The mode for out degree is 2. These two numbers tells us that the most common in- and out-degree is relatively low, considering the total number of nodes. The minimum in- and out-degree are both 0, an out-degree of 0 corresponds to a character that links to no other characters, but is linked to by at least one other character. The maximum in-degree is 429, this is the character that most other characters are linking to, corresponding to roughly 1/4 of all of our nodes linking to this single node. The maximum out degree is 68, which corresponds to the character that has the most links to other characters. Overall these numbers indicates to us that the graph is not very dense, since it looks like we (might) have quite a few nodes that do not have many edges going in or out of them. At the same time this also indicates that we must have some nodes that have many links, relatively to the others. 

**We also want the degree distributions and a plot of the network**
- **Create and visualize in- and out-going degree distributions as described in Lecture 4. Think about which axes you should use - loglog is great for power-law distributions, but not necessarily for Poisson-like degree distributions**

In [ ]:
# Creating the degree distirbution for in and out degress
in_counts, in_bins = np.histogram(in_degree_list, bins=max_in_degree)
out_counts, out_bins = np.histogram(out_degree_list, bins=max_out_degree)

#Plotting the in-degree distribution with log-log axes 
#as the in-degree distribution is much like a power-law distribution
plt.plot(in_bins[:-1], in_counts, color = 'green')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Degree')
plt.ylabel('Number of Nodes')
plt.title('In-degree distributions')
plt.show()

#Plotting the out-degree distribution with normal axes 
#as the out-degree distribution is much like a Poisson distribution
plt.plot(in_bins[:-1], in_counts, color = 'green')
plt.plot(out_bins[:-1], out_counts, color = 'red')
plt.xlabel('Degree')
plt.ylabel('Number of Nodes')
plt.title('Out-degree distributions')
plt.show()

- **Estimate, report, and reflect on the slope and starting value of the incoming degree distribtion's power law using the tools described in Lecture 5 - and display the fit on top of your incoming degree-distribution plot**

In [ ]:
power_law = powerlaw.Fit(in_degree_list)
print(power_law.alpha)

> From the value of alpha we can see that the in-degrees belong to a power-law distribution. 

- **Plot the network using the Force Atlas algorithm as described in Lecture 5.**

In [ ]:
# Lists to contain nodes belonging to DC and Marvel
dc_nodes = []
marvel_nodes = []

#Lists to contain the degrees for the DC nodes and the Marvel nodes. To be used when plotting using the Force Atlas
dc_degrees = []
marvel_degrees = []

# Dividing nodes in Marvel and DC universe
for n in uG.nodes():
    if uG.nodes[n]['universe'] == 'Marvel':
        marvel_nodes += [n]    
        marvel_degrees += [uG.degree[n]]
    else:
        dc_nodes += [n]
        dc_degrees += [uG.degree[n]] 

In [ ]:
forceatlas2 = ForceAtlas2(
                        # Behavior alternatives
                        outboundAttractionDistribution=True,  # Dissuade hubs
                        linLogMode=False,  # NOT IMPLEMENTED
                        adjustSizes=False,  # Prevent overlap (NOT IMPLEMENTED)
                        edgeWeightInfluence=1.0,

                        # Performance
                        jitterTolerance=1.0,  # Tolerance
                        barnesHutOptimize=True,
                        barnesHutTheta=1.2,
                        multiThreaded=False,  # NOT IMPLEMENTED

                        # Tuning
                        scalingRatio=2.0,
                        strongGravityMode=False,
                        gravity=0.5,

                        # Log
                        verbose=True)

positions = forceatlas2.forceatlas2_networkx_layout(G, pos=None, iterations=2000)
nx.draw_networkx_nodes(G, positions, nodelist = marvel_nodes, node_size=marvel_degrees, node_color="blue", alpha=0.4)
nx.draw_networkx_nodes(G, positions, nodelist = dc_nodes, node_size=dc_degrees, node_color="red", alpha=0.4)
nx.draw_networkx_edges(G, positions, edge_color="green", alpha=0.05)
plt.axis('off')
plt.show()

# Communities

**Identify the communities in one or both of the superhero universes (DC/Marvel) as described in Week 7.**

> We will be focusing on the DC superhero universe. We start by recreating our graph for just the DC characters:

In [ ]:
# Recreating the network H, for only the dc universe
dc_files = os.listdir('dc')
H = nx.DiGraph()

dc_nodes = []
marvel_nodes = []

# For week 5 we are adding the lengths of the wikitext for each character
for dc_char in dc_files:
  H.add_node(dc_char[:-4], uni="dc", length=0)

# Adding all the dc nodes and edges
for source in dc_files:
  # Remove .txt ending for character name
  name = source[0:len(source)-4]

  # Get the wikitext saved for this character
  with open('dc/' + source) as f:
    lines = f.read()

  # Find all the links in this wikitext
  matches = re.findall(r'[[]{2}.*?[]]{2}', lines)

  # Count the number of words in this wikitext and set that attribute
  word_count = len(re.findall(r'\w+', lines))
  H.nodes[name]['length'] = word_count

  # Construct possible file name:
  links = []
  for match in matches: 
    link = match[2:len(match)-2]
    link = link.split("|")[0]
    link = re.sub(r'\s', '_', link) 
    #link.replace("ñ", "n")
    link.replace("/", "-")
    links.append(link)

  # Remove duplicates
  links = [*set(links)]

  # Add an edge for each link that can be found in either fold
  for link in links:
    if os.path.isfile('dc/' + link + ".txt") or os.path.isfile('marvel/' + link + ".txt"):
      H.add_edge(name, link)

largest_cc = max(nx.weakly_connected_components(H), key=len)
gcc = nx.subgraph(H, largest_cc)
uH = gcc.to_undirected(reciprocal=False, as_view=True)

In [ ]:
#Creating the DC network
dc_G = uG.subgraph(dc_nodes)

In [ ]:
# Example from: https://perso.crans.org/aynaud/communities/
#first compute the best partition
partition = community_louvain.best_partition(dc_G)


partition
# Some code to find the number of communities and the number of nodes in each, needed for next question

#set(partition.values())

In [ ]:
#Not staying in assignment, just to see size of communitites and which colors they are getting
for com in set(partition.values()) :
 #   count += 1
    #print("com: " + str(com))
    #print("count: " + str(count))
    list_nodes = [nodes for nodes in partition.keys() if partition[nodes] == com]
    print(len(list_nodes))
cmap

**Visualize the communities by coloring the graph's nodes according to community affiliation - also as described in Week 7.**

In [ ]:
#drawing using ForceAtlas
forceatlas2 = ForceAtlas2(
                        # Behavior alternatives
                        outboundAttractionDistribution=True,  # Dissuade hubs
                        linLogMode=False,  # NOT IMPLEMENTED
                        adjustSizes=False,  # Prevent overlap (NOT IMPLEMENTED)
                        edgeWeightInfluence=1,

                        # Performance
                        jitterTolerance=5.0,  # Tolerance
                        barnesHutOptimize=True,
                        barnesHutTheta=0.5,
                        multiThreaded=False,  # NOT IMPLEMENTED

                        # Tuning
                        scalingRatio=3.0,
                        strongGravityMode=True,
                        gravity=0.5,

                        # Log
                        verbose=True)

positions = forceatlas2.forceatlas2_networkx_layout(dc_G, pos=None, iterations=2000)
cmap = plt.get_cmap('plasma', max(partition.values()) + 1)

nx.draw_networkx_nodes(dc_G, positions, partition.keys(), node_size=40,
                        cmap=cmap, node_color=list(partition.values()))
nx.draw_networkx_edges(dc_G, positions, edge_color='green', alpha=0.5)
plt.show()


**Write a paragraph about your work to identify the communities. The paragraph should include**
- **Information about the algorithm you used to find communities.**
- **The value of modularity for your network.**
- **The number of communities and their sizes (in terms of number of nodes).**

> This is just a placeholder for the paragraph. Something about the Louvain algorithm. Something about modularity. Something about the number of communities and their sizes.

**For the 10 largest communities, process the text in order to create a TF-IDF vector for each community. Explain in your own words how TF-IDF works.**

> This is just a placeholder for the paragraph explaining how TF-IDF works

**Then visualize the TF-IDF vectors using wordclouds (as described in Week 7). Remember to comment on your word-clouds (e.g. in the figure captions): Do the wordclouds/TF-IDF lists enable you to understand the communities you have found (or is it just gibberish)? Justify your answer.**

> This is just a placeholder for the paragraph for our answer and the justification

# Sentiment

**First calculate the sentiment for all character pages. Calculate the mean value of the sentiment and the 10th and 90th percentile values for the sentiment. Then create a nice histogram displaying the distribution of sentiment values for all pages. Indicate the mean, etc on that histogram.**

> The following code shows how we have calculated and found sentiments for the different files, but since we had to store the graph as an edgelist, this had to be done before we got to this part of the assignment.
We start by preparing the data from the files that have been given for the exercise. First we make a list of all the words from the given sentiment list with their associated average happiness score:

In [ ]:
# Create an empty list to store tuples of words and their 
# average happiness score
sent_list = []

# Read in the .tsv file
with open("dataset_s1.tsv", "r", encoding="utf8") as sent_file:
    tsv_reader = csv.DictReader(sent_file, delimiter="\t")
    # For each sentiment in the file, save the word and average happiness in a tuple
    # and add it to the list
    for sent in tsv_reader:
        word = sent["word"]
        average = sent["happiness_average"]
        sent_list.append((word, average))

> Then we create a function to calculate the sentiment from a frequency distribution for a given file:

In [ ]:
# Function for calculating the sentiment of a file from the frequency distribution for that file
def sentiment(tokens):
    # Total sentiment score of file
    sent_sum = 0.0
    # Total number of occurences of words
    occ_sum = 0
    
    # For each token and associated number of occurences
    for token, occ in tokens.items():
        # If the token is in the given list of words with rated happiness
        for word, score in sent_list:
            if token == word:
                sent_sum += (float(score) * occ)
                occ_sum += occ
    return sent_sum / occ_sum

> We have added an attribute sentiment to the nodes, and as the other attributes were set for each node, the sentiment for that node was calculated using the above `sentiment`function. We then create a list of tuples, where each tuple contains a character name and the associated sentiment:

In [ ]:
sentiments = nx.get_node_attributes(G, "sentiment")

> We are now ready to calculate the mean, 10th percentile, and the 90th percentile:

In [ ]:
#Source: https://stackoverflow.com/questions/2374640/how-do-i-calculate-percentiles-with-python-numpy
# mean
mean_total = np.mean(list(sentiments.values()))

# 10th percentile
per_10th = np.percentile(list(sentiments.values()), 10)

# 90 percentile sentiment 
per_90th =  np.percentile(list(sentiments.values()), 90)

> We are now able to make the histogram with the distribution and the calculated values:

In [ ]:
# Printing the calculated values
print(f"Calculated values:")
print(f"mean: {mean_total}")
print(f"10th percentile: {per_10th}")
print(f"90th percentile: {per_90th}")

# Setting up figure with histogram
result = plt.hist(list(sentiments.values()), bins=20, color='c', edgecolor='k', alpha=0.65)
plt.axvline(per_10th, color='blue', linestyle='dashed', linewidth=1, label="10th percentile")
plt.axvline(per_90th, color='green', linestyle='dashed', linewidth=1, label="90th percentile")
plt.axvline(mean_total, color='orange', linestyle='dashed', linewidth=1, label="mean")
plt.xlabel('sentiment')
plt.ylabel('counts')
plt.figtext(.5,-0.05, f"Histogram displaying the distribution of sentiments for all of the characters from the Marvel and DC universes. The dashed blue line is the 10th percentile, the dashed green line is the 90th percentile, and the dashed orange line is the mean.", ha='center', fontsize=6)
plt.legend()
plt.show()

> From the histogram and the calculated values it is clear to see that the overall sentiment is pretty low. We would have expected that the sentiment would be centered around 5, or maybe a bit lower, reflecting that the wikipedia articles strive to have a neutral language. However, given that most super hero comics are action oriented, it may be that they have more words with a lower happiness score in their articles from titles of issues, movies, and character names. 

**Now grab the good/bad character names (from this file, see Week 8 for details) and calculate the same stats for only good/bad characters; also plot the histograms for the good/bad group and indicate the values of the mean, etc.**

> We start by creating lists of the good and the bad guys. Since there may be discrepancies between the characters in our graph and the provided file, we make sure to check for this when creating these lists, leaving out characters from our graph that are not represented in the given file and vice versa. Furthermore we have to take into account that some of the characters in the provided file are neither good nor bad, we have decided to take out these characters as well, since we would not be able to tell which group to count them in:

In [ ]:
# Read in the given file
with open('wiki_meta_data.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

# Initialize lists to hold the names of the good and bad guys
good_guys = []
bad_guys = []
characters = []
alignments = []


# Prepare a list of the characters in our graph
characters_in_graph = list(G.nodes())
  
# For each character in the given file
for info in data:
    # Get the name using the same format as our nodes
    name = info[1].replace(" ", "_")
    # Check if the character from the file is in our graph
    if name in characters_in_graph:
        # If the character is good
        if info[6] == '1':
            # Add that character name to the list of good guys
            good_guys.append(name)
            # Add the current alignment for that index in the alignment list
            alignments.append(1)  
            # Add the character to the list of characters
            characters.append(name)
        # If the character is bad
        if info[7] == '1':
            # Add that character name to the list of bad guys
            bad_guys.append(name)
            # Add the current alignment for that index in the alignment list
            alignments.append(0)
            # Add the character to the list of characters
            characters.append(name)

> We can now go through our graph and grab the previously calculated sentiments for each node, and add it to a list of either sentiments for good or bad guys:

In [ ]:
# Init lists to hold sentiment values for good guys (gg) and bad guys (bg)
gg_sent = []
bg_sent = []

# For each tuple consisting of a character name (node) and a sentiment value in the all_nodes list
for char in characters:
    # If the character is one of the good guys
    if char in good_guys:
        # Add its sentiment value to the list of sentiments for good guys
        gg_sent.append(G.nodes[char]["sentiment"])
    # If the character is one of the bad guys
    if char in bad_guys:
        # Add its sentiment value to the list of the sentiments for the bad guys
        bg_sent.append(G.nodes[char]["sentiment"])  

> The mean, 10th and 90th percentiles can now be calculated for both good and bad guys, and plotted in histograms:

In [ ]:
# Histogram for good and bad guys
gg_counts, gg_bins = np.histogram(gg_sent)
bg_counts, bg_bins = np.histogram(bg_sent)

# Setting same number of bins for both
number_of_bins = 10 

# Create two subplots and unpack the output array immediately
fig, axs = plt.subplots(1, 2, sharex=True)
fig.set_figwidth(15)

# Histogram for good guys
histogram = np.histogram(gg_sent, number_of_bins)

# Calculate values
mean_gg = np.mean(gg_sent) # Insert data
per_10th_gg = np.percentile(gg_sent, 10) # Insert data
per_90th_gg =  np.percentile(gg_sent, 90) # Insert data

axs[0].hist(gg_sent, bins = 25, edgecolor='black')
axs[0].set_title(f'Histogram of good guys sentiment')
axs[0].set_xlabel('sentiment')
axs[0].set_ylabel('counts')
axs[0].axvline(per_10th_gg, color='blue', linestyle='dashed', linewidth=1, label="10th percentile")
axs[0].axvline(per_90th_gg, color='green', linestyle='dashed', linewidth=1, label="90th percentile")
axs[0].axvline(mean_gg, color='orange', linestyle='dashed', linewidth=1, label="mean")
axs[0].legend()

# Histogram for bad guys
histogram = np.histogram(bg_sent, number_of_bins)

# Calculate values for plot
mean_bg = np.mean(bg_sent) # Insert data
per_10th_bg = np.percentile(bg_sent, 10) # Insert data
per_90th_bg =  np.percentile(bg_sent, 90) # Insert data

axs[1].hist(bg_sent, bins = 25, edgecolor='black')
axs[1].set_title(f'Histogram of bad guys sentiment')
axs[1].set_xlabel('sentiment')
axs[1].set_ylabel('counts')
axs[1].axvline(per_10th_bg, color='blue', linestyle='dashed', linewidth=1, label="10th percentile")
axs[1].axvline(per_90th_bg, color='green', linestyle='dashed', linewidth=1, label="90th percentile")
axs[1].axvline(mean_bg, color='orange', linestyle='dashed', linewidth=1, label="mean")
plt.figtext(.5,-0.05, f"Left plot: Histogram displaying the distribution of sentiments for all of the good guys from both universes. The dashed blue line is the 10th percentile, the dashed green line is the 90th percentile, and the dashed orange line is the mean.\nRight plot: Histogram displaying the distribution of sentiments for all of the bad guys from both universes. The dashed blue line is the 10th percentile, the dashed green line is the 90th percentile, and the dashed orange line is the mean.", ha='center', fontsize=8)
axs[1].legend()

# Print the calculated values
print(f"Good guys:\t\t\t\tBad guys:")
print(f"mean: {mean_gg}\t\tmean: {mean_bg}")
print(f"10th percentile: {per_10th_gg}\t10th percentile: {per_10th_bg}")
print(f"90th percentile: {per_90th_gg}\t90th percentile: {per_90th_bg}")

# Show both histrograms
plt.tight_layout()
plt.show()

> Again we see that overall the sentiments for both good and bad guys are pretty low. There seems to be a slight indication that bad guys actually have a higher sentiment than good guys, if we look at the calculated values.

**Finally use the label permutation test (see Week 5) to investigate the two hypotheses below**
- **H1: Good characters have a higher averge value for sentiment than a similarly sized set of randomly selected characters.**
- **H2: Bad characters have a lower average value for sentiment than a similarly sized set of randomly selected characters.**

> For the label permutation test we are using two lists. `characters` contains the character names and `alignments` contains 1's and 0's, a 1 indicates good and 0 indicates bad. The indeces of lists corrspond to the same character, such that the i'th character in `characters` has the alignment of the i'th index of `alignments`. This allows us to just make a random sample in `alignments` to reshuffle the labels for each character, since there will always be the same fraction of good and bad labels, and the character names are kept at the same indeces in the `characters` list:

In [ ]:
# Init lists to hold the values for each sample generated
average_sentiment_samples_good_guys = []
average_sentiment_samples_bad_guys = []

# Create 1000 samples
for n in range(1000):
    # We want to start by shuffling who is bad and good.
    alignments = random.sample(alignments, k=len(alignments))
    
    # We can then get the alignment of the nodes by finding their index in the characters list
    # and then looking up their (shuffled) alignment in the alignments list
    sent_bad = 0
    sent_good = 0
    
    #rem_c = []
    
    # Since we are taking a sample in the size of all the characters we have
    # we just go through all of the characters and check their sentiment and
    # newly assigned alignment (good/bad)
    for c in characters:
        a = alignments[characters.index(c)]
        if a == 1: # if its a good guy
            sent_good += G.nodes[c]["sentiment"]
        if a == 0: # if its a bad guy
            sent_bad += G.nodes[c]["sentiment"]
    
    # Divide the sum by the number of good/bad guys (in this sample)
    # Add this average to the average_sentiment_samples list
    average_sentiment_samples_good_guys.append(sent_good / len(good_guys))
    average_sentiment_samples_bad_guys.append(sent_bad / len(bad_guys))

# Plot a histogram of the averages
fig, axs = plt.subplots(1, 2, sharex=True, figsize=(15, 5))
axs[0].hist(average_sentiment_samples_good_guys, bins = 10, edgecolor='black')
axs[0].axvline(mean_gg, color='orange', linestyle='dashed', linewidth=1, label="mean")
axs[0].set_title('Result of label permutation for good guys')
axs[0].legend()

axs[1].hist(average_sentiment_samples_bad_guys, bins = 10, edgecolor='black')
axs[1].axvline(mean_bg, color='orange', linestyle='dashed', linewidth=1, label="mean")
axs[1].set_title('Result of label permutation for Bad guys')
axs[1].legend()

plt.figtext(.5,-0.05, f"Left plot: Histogram displaying the distribution of sentiments from the label permutation for the good guys from both universes. The dashed orange line is the mean..\nLeft plot: Histogram displaying the distribution of sentiments from the label permutation for the bad guys from both universes. The dashed orange line is the mean.", ha='center')

plt.show()

**Write a short paragraph reflecting on your findings.**

> Our findings seem to further support what we observed when we were just looking at the good and bad guys previously. Namely that the good guys have a lower sentiment than the average character, and the bad guys have a higher sentiment than the average character. So based on this we would have to reject both H1 and H2, which seems a bit off. While this may simply be a case of the data that we have used, we want to make an argument for this and a tiny investigation into this:

**Bonus paragraphs: Argument and investigation**

> Argument: On one hand we suspect that this may not comepletely unlikely that the opposite of H1 and H2 is the case. Even though most heroes are more well documented with love affairs, families etc, they are also more likely to have a "sad" origin story, and have more well documented deaths of other characters around them. Since the heroes are usually the main characters they also appear in more stories, and as stated previously many of these stories may have titles containing "sad" words such as war, weapon, fight etc. On the other hand the villains may be less well documented, and appear in fewer comics. As such a villain that has its origin in an origin story love, fantasy, or protection of something, may get a higher sentiment, simply because it has fewer words, and a larger fraction of them are "happy". We are not comic book experts, so these are just some thoughts that might explain what we saw above.

In [ ]:
# Are heroes more well documented than villains?
gg_doc = []
bg_doc = []

for char in characters:
    if char in good_guys:
        gg_doc.append(G.nodes[char]["txtlength"])
    if char in bad_guys:
        bg_doc.append(G.nodes[char]["txtlength"])

# Calculate mean, 10th perc, 90th perc for good guys
mean_gg = np.mean(gg_doc) 
per_10th_gg = np.percentile(gg_doc, 10)
per_90th_gg =  np.percentile(gg_doc, 90)

# Calculate mean, 10th perc, 90th perc for bad guys
mean_bg = np.mean(bg_doc) 
per_10th_bg = np.percentile(bg_doc, 10)
per_90th_bg =  np.percentile(bg_doc, 90) 

# Print the calculated values
print(f"Good guys:\t\t\tBad guys:")
print(f"mean: {mean_gg}\t\tmean: {mean_bg}")
print(f"10th percentile: {per_10th_gg}\t\t10th percentile: {per_10th_bg}")
print(f"90th percentile: {per_90th_gg}\t90th percentile: {per_90th_bg}")

> So the good guys are better documented than the bad guys, in that they have longer texts describing them. Do they also have a higher fraction of negative words? Lets say the average happyness value for neutral is 5, then we count how large a fraction of the words for each character are negative and how many are positive:

In [ ]:
# functions
def frac_sad(tokens):
    # Total sentiment score of file
    sent_sum = 0.0
    # Total number of occurences of words
    occ_sum = 0
    
    # For each token and associated number of occurences
    for token, occ in tokens.items():
        # If the token is in the given list of words with rated happiness
        for word, score in sent_list:
            if token == word and float(score) < 5:
                sent_sum += occ
        occ_sum += occ
    return sent_sum / occ_sum

def frac_hap(tokens):
        # Total sentiment score of file
    sent_sum = 0.0
    # Total number of occurences of words
    occ_sum = 0
    
    # For each token and associated number of occurences
    for token, occ in tokens.items():
        # If the token is in the given list of words with rated happiness
        for word, score in sent_list:
            if token == word and float(score) > 5:
                sent_sum += occ
        occ_sum += occ
    return sent_sum / occ_sum

In [ ]:
gg_sad = []
bg_sad = []
gg_hap = []
bg_hap = []
uncommented = 0

# The code below has been commented out, since it takes a while to execute,
# the following 2 code cells are using the code that has been commented out
# in this cell. Hence, if this code is not uncommented the other will not 
# work properly, but we have written about our findings.

#for file in os.listdir("dc/"):
#    orig_character = file[:-4]
#    fileopener = open("dc/" + file, 'r')
#    text = fileopener.read()  
#    tokens = nltk.wordpunct_tokenize(BeautifulSoup(text, 'html.parser').get_text())
#    file_text = [w.lower() for w in tokens if w.isalpha()]
#    fdist = FreqDist(file_text)
#    if orig_character in good_guys:
#        gg_sad.append(frac_sad(fdist))
#        gg_hap.append(frac_hap(fdist))
#    if orig_character in bad_guys:
#        bg_sad.append(frac_sad(fdist))
#        bg_hap.append(frac_hap(fdist))
#       
#for file in os.listdir("marvel/"):
#    orig_character = file[:-4]
#    fileopener = open("marvel/" + file, 'r')
#    text = fileopener.read()  
#    tokens = nltk.wordpunct_tokenize(BeautifulSoup(text, 'html.parser').get_text())
#    file_text = [w.lower() for w in tokens if w.isalpha()]
#    fdist = FreqDist(file_text)
#    if orig_character in good_guys:
#        gg_sad.append(frac_sad(fdist))
#        gg_hap.append(frac_hap(fdist))
#    if orig_character in bad_guys:
#        bg_sad.append(frac_sad(fdist))
#        bg_hap.append(frac_hap(fdist))
#
#uncommented = 1

In [ ]:
if uncommented == 1:
    # Calculate mean, 10th perc, 90th perc for good guys
    mean_gg = np.mean(gg_sad) 
    per_10th_gg = np.percentile(gg_sad, 10)
    per_90th_gg =  np.percentile(gg_sad, 90)

    # Calculate mean, 10th perc, 90th perc for bad guys
    mean_bg = np.mean(bg_sad) 
    per_10th_bg = np.percentile(bg_sad, 10)
    per_90th_bg =  np.percentile(bg_sad, 90) 

    # Print the calculated values
    print(f"Good guys:\t\t\t\tBad guys:")
    print(f"mean: {mean_gg}\t\tmean: {mean_bg}")
    print(f"10th percentile: {per_10th_gg}\t10th percentile: {per_10th_bg}")
    print(f"90th percentile: {per_90th_gg}\t90th percentile: {per_90th_bg}")

In [ ]:
if uncommented == 1:
    # Calculate mean, 10th perc, 90th perc for good guys
    mean_gg = np.mean(gg_hap) 
    per_10th_gg = np.percentile(gg_hap, 10)
    per_90th_gg =  np.percentile(gg_hap, 90)

    # Calculate mean, 10th perc, 90th perc for bad guys
    mean_bg = np.mean(bg_hap) 
    per_10th_bg = np.percentile(bg_hap, 10)
    per_90th_bg =  np.percentile(bg_hap, 90) 

    # Print the calculated values
    print(f"Good guys:\t\t\t\tBad guys:")
    print(f"mean: {mean_gg}\t\tmean: {mean_bg}")
    print(f"10th percentile: {per_10th_gg}\t10th percentile: {per_10th_bg}")
    print(f"90th percentile: {per_90th_gg}\t90th percentile: {per_90th_bg}")

In [ ]:
# Lastly we just want to check how many characters are in the two lists
print(f"Number of good guys = {len(good_guys)}")
print(f"Number of bad guys = {len(bad_guys)}")

> We found that the the good guys had a lower fraction of "sad" words and a higher fraction of "happy" words. Which conincides with the original H1 and H2. Hence our idea for an alternative hapythesis may be incorrect. We also considered that it might have been caused by us having a much lower number of characters in one of the groups, but as can be seen above, this is not the case. 